Example EDA for IMDB TV shows:
- https://towardsdatascience.com/imdb-tv-show-data-analysis-4961ef39d174
- https://towardsdatascience.com/imdb-television-show-data-analysis-part-2-39ebf47977ff

In [1]:
# set the width to full
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import dependencies and libraries
import pandas as pd
import numpy as np

In [3]:
# set parameters to show max rows and columns and change format numbers to show three digits after decimal
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
# Source: https://datasets.imdbws.com/

# read the datasets
basics = pd.read_csv('data/title.basics.tsv.gz', compression='gzip', header=0, sep='\t')
ratings = pd.read_csv('data/title.ratings.tsv.gz', compression='gzip', header=0, sep='\t')
principals = pd.read_csv('data/title.principals.tsv.gz', compression='gzip', header=0, sep='\t')
episodes = pd.read_csv('data/title.episode.tsv.gz', compression='gzip', header=0, sep='\t')
names = pd.read_csv('data/name.basics.tsv.gz', compression='gzip', header=0, sep='\t')
crews = pd.read_csv('data/title.crew.tsv.gz', compression='gzip', header=0, sep='\t')
akas = pd.read_csv('data/title.akas.tsv.gz', compression='gzip', header=0, sep='\t')

In [5]:
basics.groupby('titleType').count()

,tconst,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
titleType,,,,,,,,
audiobook,1,1,1,1,1,1,1,1
episode,1,1,1,1,1,1,1,1
movie,570842,570842,570842,570842,570842,570842,570842,570842
radioSeries,1,1,1,1,1,1,1,1
short,799224,799224,799224,799224,799224,799224,799224,799224
tvEpisode,5593217,5593209,5593209,5593217,5593217,5593217,5593217,5593207
tvMiniSeries,36291,36291,36291,36291,36291,36291,36291,36291
tvMovie,130437,130437,130437,130437,130437,130437,130437,130437
tvSeries,203265,203265,203265,203265,203265,203265,203265,203265


In [6]:
# first take the types of media that we are interested in
basics_interested = basics[basics.titleType.isin(['movie', 'tvMiniSeries', 'tvSeries', 'tvMovie', 'video', 'tvEpisode', 'tvSpecial', 'short'])]
#dictionary_for_mapping_mediaTypes = {'movie': "FILM", 'tvMiniSeries': "TV", 'tvSeries': "TV"}
#basics_interested["titleType"].replace(dictionary_for_mapping_mediaTypes, inplace=True)
#basics_interested["primaryTitle"] = basics_interested["primaryTitle"].str.lower()

In [7]:
# read the main data [source](https://www.kaggle.com/ksb357/military-hollywood-collaboration-database)
military_hollywood = pd.read_csv('military-hollywood-full_imdbidAdded.csv', delimiter=',', usecols=['Title', 'IMDB_ID', 'Status', 'Media Type', 'Year','Remarks'])

In [8]:
military_hollywood.shape

(858, 6)

In [9]:
military_hollywood.head()

,Title,IMDB_ID,Status,Media Type,Year,Remarks
0,"""1968""",NaN,OTH,FILM,NaN,THE FILM STARTED OUT VERY NEGATIVE FOR THE ARM...
1,"1,000 MEN AND A BABY",tt0133231,APP,TV,1997.0,VERY POSITIVE DEPICTION OF NAVY IN THIS KOREAN...
2,1ST FORCE,NaN,OTH,FILM,NaN,INITIALLY DOD AND USMC WERE INCLINED TO SUPPOR...
3,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...
4,3RD DEGREE,tt0098469,APP,TV,1989.0,PERSONNEL APPEARED ON THIS GAME SHOW AT THE EX...


In [10]:
# join main dataset and title basics dataset for release year, genres, and title type informations
aggregated_data = pd.merge(military_hollywood, basics_interested[['tconst', 'startYear', 'genres', 'titleType']], how ='left', left_on='IMDB_ID', right_on='tconst')

In [11]:
# check if any rows with imdb_id not null but tconst is null
aggregated_data[(aggregated_data.IMDB_ID.notna())&(aggregated_data.tconst.isna())]

,Title,IMDB_ID,Status,Media Type,Year,Remarks,tconst,startYear,genres,titleType


In [12]:
aggregated_data[['Media Type', 'titleType']].drop_duplicates()#[(aggregated_data.IMDB_ID.notna())&(aggregated_data.tconst.isna())]

,Media Type,titleType
0,FILM,NaN
1,TV,tvMovie
3,TV,tvSeries
5,FILM,movie
15,OTH,video
23,TV,tvEpisode
26,TV,NaN
30,TV,tvSpecial
36,TV,short
43,FILM,tvSeries


In [13]:
# join merged dataset above and crews dataset for directors informations 
aggregated_data1 = pd.merge(aggregated_data, crews[['tconst', 'directors']], how ='left', left_on='IMDB_ID', right_on='tconst')

In [14]:
# create a dictionary with director names and their ids
df_director = names[['nconst', 'primaryName']].set_index('nconst')
dict_director = df_director.to_dict()
dict_director = dict_director['primaryName']

In [16]:
# divide data into 2 to apply the dictionary defined above
aggregated_data1_directors_isna = aggregated_data1.loc[aggregated_data1.directors.isna()]
aggregated_data1_directors_notna = aggregated_data1[~aggregated_data1.index.isin(aggregated_data1_directors_isna.index)]

In [18]:
# for the directors information of more than one directors split them w.r.t. ',' and then explode the directors column
aggregated_data1_directors_notna.directors = [x.split(',') for x in aggregated_data1_directors_notna.directors]
aggregated_data1_directors_notna = aggregated_data1_directors_notna.explode('directors')
aggregated_data1_directors_notna.head(20)

,Title,IMDB_ID,Status,Media Type,Year,Remarks,tconst_x,startYear,genres,titleType,tconst_y,directors
1,"1,000 MEN AND A BABY",tt0133231,APP,TV,1997.0,VERY POSITIVE DEPICTION OF NAVY IN THIS KOREAN...,tt0133231,1997,Drama,tvMovie,tt0133231,nm0170680
3,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...,tt0285331,2001,"Action,Crime,Drama",tvSeries,tt0285331,nm1879589
3,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...,tt0285331,2001,"Action,Crime,Drama",tvSeries,tt0285331,nm0514546
3,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...,tt0285331,2001,"Action,Crime,Drama",tvSeries,tt0285331,nm0320987
3,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...,tt0285331,2001,"Action,Crime,Drama",tvSeries,tt0285331,nm0788610
3,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...,tt0285331,2001,"Action,Crime,Drama",tvSeries,tt0285331,nm0143984
3,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...,tt0285331,2001,"Action,Crime,Drama",tvSeries,tt0285331,nm0346550
3,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...,tt0285331,2001,"Action,Crime,Drama",tvSeries,tt0285331,nm0463926
3,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...,tt0285331,2001,"Action,Crime,Drama",tvSeries,tt0285331,nm0388182
3,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...,tt0285331,2001,"Action,Crime,Drama",tvSeries,tt0285331,nm0877427


In [19]:
# apply the dictionary defined above to have the name of the directors
aggregated_data1_directors_notna['directors'].replace(dict_director, inplace=True)

def join_(str):
    """
    joins given strings with comma
    """
    return ','.join(str)

# aggregate the exploded data after replacing the directors' ids with their names
aggregated_data1_directors_notna = pd.pivot_table(aggregated_data1_directors_notna, values="directors", index=['Title', 'IMDB_ID', 'Status', 'Media Type', 'Year', 'Remarks','tconst_x', 'startYear', 'genres', 'titleType', 'tconst_y'], 
               aggfunc=join_).reset_index()

aggregated_data1_directors_notna.head(20)

,Title,IMDB_ID,Status,Media Type,Year,Remarks,tconst_x,startYear,genres,titleType,tconst_y,directors
0,"1,000 MEN AND A BABY",tt0133231,APP,TV,1997.0,VERY POSITIVE DEPICTION OF NAVY IN THIS KOREAN...,tt0133231,1997,Drama,tvMovie,tt0133231,Marcus Cole
1,24,tt0285331,APP,TV,2004.0,APPROVED FILMING FOR ONE DAY WITH TWO MARINE C...,tt0285331,2001,"Action,Crime,Drama",tvSeries,tt0285331,"Nelson McCormick,Dwight H. Little,Ken Girotti,..."
2,3RD DEGREE,tt0098469,APP,TV,1989.0,PERSONNEL APPEARED ON THIS GAME SHOW AT THE EX...,tt0098469,1989,"Crime,Drama,Thriller",tvMovie,tt0098469,Roger Spottiswoode
3,55 DAYS AT PEKING,tt0056800,OTH,FILM,1963.0,PERIOD PIECE ABOUT MARINES IN CHINA DURING THE...,tt0056800,1963,"Action,Adventure,Drama",movie,tt0056800,"Guy Green,Nicholas Ray"
4,84 CHARLIE MOPIC,tt0096744,OTH,FILM,1989.0,VIETNAM MOVIE ABOUT MOTION PICTURE CORRESPONDE...,tt0096744,1989,"Drama,War",movie,tt0096744,Patrick Sheane Duncan
5,A FEW GOOD MEN,tt0104257,LIM,FILM,1992.0,"INACCURATE, NEGATIVE PORTRAVALS OF MARINES. PR...",tt0104257,1992,"Drama,Thriller",movie,tt0104257,Rob Reiner
6,A MIDNIGHT CLEAR,tt0102443,DEN,FILM,1992.0,DECLINED ASSISTANCE (REQUEST FOR WW II FACILIT...,tt0102443,1992,"Drama,War",movie,tt0102443,Keith Gordon
7,ABOVE AND BEYOND,tt0044324,APP,FILM,1953.0,STORY OF PAUL TIBBETTS AND THE ATOMIC BOMB MIS...,tt0044324,1952,"Action,Biography,Drama",movie,tt0044324,"Melvin Frank,Norman Panama"
8,ABOVE THE CLOUDS,tt0024811,APP,FILM,1934.0,"NAVY PROVIDED USE OF BATTLESHIPS, SUBMARINES, ...",tt0024811,1933,"Action,Adventure,Drama",movie,tt0024811,Roy William Neill
9,ACE OF ACES,tt0023737,APP,FILM,1933.0,ARMY AIR CORPS PROVIDED PLANES TO STAGE AERIAL...,tt0023737,1933,"Drama,War",movie,tt0023737,J. Walter Ruben


In [20]:
# combine the divided data back together
aggregated_data2 = aggregated_data1_directors_notna.append(aggregated_data1_directors_isna, ignore_index=True)
# keep only necessary fields
aggregated_data2 = aggregated_data2[['Title', 'IMDB_ID', 'Status', 'Media Type', 'Year', 'Remarks', 'startYear', 'genres', 'titleType', 'directors']]

In [21]:
# join the aggregated data 2 with ratings to have the average rating information
aggregated_data3 = pd.merge(aggregated_data2, ratings[['tconst', 'averageRating']], how ='left', left_on='IMDB_ID', right_on='tconst')

In [22]:
from imdb import IMDb
ia = IMDb()

In [27]:
# create two subsets of the aggregated_data3 to be able to request plot and keywords information for each imdb_id
aggregated_data3_imdbid_isna = aggregated_data3.loc[(aggregated_data3.IMDB_ID.isna()) | (aggregated_data3.IMDB_ID=='')]
aggregated_data3_imdbid_notna = aggregated_data3[~aggregated_data3.index.isin(aggregated_data3_imdbid_isna.index)]

In [28]:
aggregated_data3_imdbid_isna['keywords'] = ''
aggregated_data3_imdbid_isna['plot'] = ''

TO DO : 

In [ ]:
aggregated_data3_imdbid_notna['keywords'] = [ia.get_movie(imdb_id_[2:], info='keywords') for imdb_id_ in aggregated_data3_imdbid_notna.IMDB_ID]
aggregated_data3_imdbid_notna['plot'] = [ia.get_movie(imdb_id_[2:], info='plot') for imdb_id_ in aggregated_data3_imdbid_notna.IMDB_ID]

In [29]:
ia.get_movie('0133231', info='plot')

<Movie id:0133231[http] title:_None_>

In [37]:
aggregated_data3_imdbid_notna.IMDB_ID[0][2:]

'0133231'

Source: https://www.kaggle.com/stephanerappeneau/350-000-movies-from-themoviedborg

In [38]:
#movies = pd.read_csv('data/AllMoviesDetailsCleaned.csv', encoding='utf-8-sig', sep=';', engine='python', parse_dates=['release_date'])

In [30]:
#movies.head()